# Understanding and Coding the Self-Attention Mechanism of Large Language Models From Scratch

Trong bài viết này, chúng ta sẽ tìm hiểu cách hoạt động của **self-attention** từ đầu. Điều này có nghĩa là chúng ta sẽ tự lập trình nó từng bước một.

Kể từ khi được giới thiệu qua bài báo gốc về **transformer** (“Attention Is All You Need”), **self-attention** đã trở thành một nền tảng của nhiều mô hình học sâu tiên tiến nhất, đặc biệt là trong lĩnh vực **Xử lý Ngôn ngữ Tự nhiên** (**Natural Language Processing - NLP**). Vì **self-attention** hiện diện ở khắp mọi nơi, nên việc hiểu cách thức hoạt động của nó là rất quan trọng.

<div style="text-align: center;">
    <img src="https://sebastianraschka.com/images/blog/2023/self-attention-from-scratch/transformer.png" alt="Transformer Diagram" width="700">
</div>


## Self-Attention

Khái niệm "attention" trong học sâu bắt nguồn từ nỗ lực cải thiện **Mạng nơ-ron hồi quy** (**Recurrent Neural Networks - RNNs**) để xử lý các chuỗi hoặc câu dài hơn. Ví dụ, hãy xem xét việc dịch một câu từ ngôn ngữ này sang ngôn ngữ khác. Việc dịch câu từng từ một không mang lại hiệu quả.

<!-- ![Sentence Translation](https://sebastianraschka.com/images/blog/2023/self-attention-from-scratch/sentence.png) -->
<div style="text-align: center;">
    <img src="https://sebastianraschka.com/images/blog/2023/self-attention-from-scratch/sentence.png" alt="Sentence Translation" width="700">
</div>

Để giải quyết vấn đề này, cơ chế attention đã được giới thiệu, cho phép truy cập vào tất cả các phần tử của chuỗi tại mỗi bước thời gian. Điểm mấu chốt là phải chọn lọc và xác định từ nào quan trọng nhất trong một ngữ cảnh cụ thể. Năm 2017, kiến trúc **transformer** đã giới thiệu một cơ chế **self-attention** độc lập, loại bỏ hoàn toàn sự phụ thuộc vào RNNs.

(Để ngắn gọn và tập trung vào các chi tiết kỹ thuật về self-attention, tôi đã bỏ qua một số phần của motivation, nhưng sách *Machine Learning with PyTorch and Scikit-Learn* của tôi có thêm thông tin chi tiết trong Chương 16 nếu bạn quan tâm.)

<p><a href="https://arxiv.org/abs/1706.03762"><img src="https://sebastianraschka.com/images/blog/2023/self-attention-from-scratch/paper.png" alt="Attention Is All You Need" style="display: block; margin-left: auto; margin-right: auto; width: 50%;"></a></p>

Chúng ta có thể coi **self-attention** là một cơ chế tăng cường nội dung thông tin của một embedding đầu vào bằng cách bao gồm thông tin về ngữ cảnh của đầu vào đó. Nói cách khác, cơ chế self-attention cho phép mô hình xác định tầm quan trọng của các phần tử khác nhau trong một chuỗi đầu vào và điều chỉnh linh hoạt ảnh hưởng của chúng lên đầu ra. Điều này đặc biệt quan trọng trong các tác vụ xử lý ngôn ngữ, nơi mà ý nghĩa của một từ có thể thay đổi dựa trên ngữ cảnh của nó trong câu hoặc tài liệu.

Lưu ý rằng có nhiều biến thể của self-attention. Đặc biệt, các nghiên cứu đã tập trung vào việc làm cho self-attention hiệu quả hơn. Tuy nhiên, hầu hết các bài báo vẫn triển khai cơ chế **attention tích vô hướng được chuẩn hóa** (**scaled-dot product attention**) gốc vì nó thường mang lại độ chính xác cao hơn, và vì self-attention hiếm khi là điểm nghẽn về mặt tính toán cho hầu hết các công ty đào tạo các mô hình transformer quy mô lớn.

Trong bài viết này, chúng ta sẽ tập trung vào cơ chế scaled-dot product attention gốc (thường được gọi là self-attention), đây vẫn là cơ chế attention phổ biến và được sử dụng rộng rãi nhất trong thực tiễn. Tuy nhiên, nếu bạn quan tâm đến các loại cơ chế attention khác, hãy xem bài khảo sát *Efficient Transformers: A Survey* năm 2020, *A Survey on Efficient Training of Transformers* năm 2023 và bài báo gần đây về **FlashAttention**.

## Embedding một câu đầu vào

Trước khi bắt đầu, chúng ta hãy xem xét một câu đầu vào “Tôi là đang làm bài tập” mà chúng ta muốn đưa qua cơ chế tự chú ý (self-attention). Tương tự như các phương pháp mô hình hóa khác để xử lý văn bản (ví dụ: sử dụng mạng nơ-ron hồi quy – recurrent neural networks hoặc mạng nơ-ron tích chập – convolutional neural networks), trước tiên chúng ta sẽ tạo một embedding cho câu này.

Để đơn giản, ở đây từ điển `dc` của chúng ta được giới hạn chỉ bao gồm các từ xuất hiện trong câu đầu vào. Trong một ứng dụng thực tế, chúng ta sẽ xem xét tất cả các từ trong tập dữ liệu huấn luyện (kích thước từ vựng điển hình dao động từ 30k đến 50k).

In [2]:
sentence = 'Tôi là đang làm bài tập,'

dc = {s:i for i,s in enumerate(sorted(sentence.replace(',', '').split()))}
print(dc)

{'Tôi': 0, 'bài': 1, 'là': 2, 'làm': 3, 'tập': 4, 'đang': 5}


In [3]:
import torch

sentence_int = torch.tensor([dc[s] for s in sentence.replace(',', '').split()])
print(sentence_int)

tensor([0, 2, 5, 3, 1, 4])


Giờ đây, sử dụng biểu diễn vector số nguyên của câu đầu vào, chúng ta có thể sử dụng một tầng embedding để mã hóa các đầu vào thành embedding dạng vector thực. Ở đây, chúng ta sẽ sử dụng một embedding 16 chiều sao cho mỗi từ đầu vào được đại diện bởi một vector 16 chiều. Vì câu gồm 6 từ, điều này sẽ dẫn đến một embedding 6×16 chiều:

In [4]:
torch.manual_seed(123)
embed = torch.nn.Embedding(6, 16)
embedded_sentence = embed(sentence_int).detach()

print(embedded_sentence)
print(embedded_sentence.shape)

tensor([[ 0.3374, -0.1778, -0.3035, -0.5880,  0.3486,  0.6603, -0.2196, -0.3792,
          0.7671, -1.1925,  0.6984, -1.4097,  0.1794,  1.8951,  0.4954,  0.2692],
        [-1.3250,  0.1784, -2.1338,  1.0524, -0.3885, -0.9343, -0.4991, -1.0867,
          0.8805,  1.5542,  0.6266, -0.1755,  0.0983, -0.0935,  0.2662, -0.5850],
        [ 0.2553, -0.5496,  1.0042,  0.8272, -0.3948,  0.4892, -0.2168, -1.7472,
         -1.6025, -1.0764,  0.9031, -0.7218, -0.5951, -0.7112,  0.6230, -1.3729],
        [ 0.8768,  1.6221, -1.4779,  1.1331, -1.2203,  1.3139,  1.0533,  0.1388,
          2.2473, -0.8036, -0.2808,  0.7697, -0.6596, -0.7979,  0.1838,  0.2293],
        [-0.0770, -1.0205, -0.1690,  0.9178,  1.5810,  1.3010,  1.2753, -0.2010,
          0.4965, -1.5723,  0.9666, -1.1481, -1.1589,  0.3255, -0.6315, -2.8400],
        [ 0.5146,  0.9938, -0.2587, -1.0826, -0.0444,  1.6236, -2.3229,  1.0878,
          0.6716,  0.6933, -0.9487, -0.0765, -0.1526,  0.1167,  0.4403, -1.4465]])
torch.Size([6, 16])


## Định nghĩa ma trận trọng số (Weight Matrices)

Bây giờ, hãy thảo luận về cơ chế tự chú ý (self-attention) được sử dụng rộng rãi, được gọi là scaled dot-product attention, tích hợp trong kiến trúc transformer.

Self-attention sử dụng ba ma trận trọng số, được gọi là $ W_q $, $ W_k $, và $ W_v $, các ma trận này được điều chỉnh như các tham số của mô hình trong quá trình huấn luyện. Các ma trận này dùng để chiếu các đầu vào thành các thành phần truy vấn (query), khóa (key), và giá trị (value) của chuỗi tương ứng.

Các chuỗi query, key và value lần lượt được lấy thông qua phép nhân ma trận giữa các ma trận trọng số $ W $ và đầu vào nhúng $ x $:

- Query sequence: $ q(i) = W_q x^{(i)} $ cho $ i \in [1, T] $
- Key sequence: $ k(i) = W_k x^{(i)} $ cho $ i \in [1, T] $
- Value sequence: $ v(i) = W_v x^{(i)} $ cho $ i \in [1, T] $

Chỉ số $ i $ biểu thị vị trí chỉ mục (index) của token trong chuỗi đầu vào, với chiều dài là $ T $.

<div style="text-align: center;">
    <img src="https://sebastianraschka.com/images/blog/2023/self-attention-from-scratch/attention-matrices.png" alt="Attention Matrices" width="400">
</div>

Ở đây, cả $ q^{(i)} $ và $ k^{(i)} $ đều là các vector có kích thước $ d_k $. Các ma trận chiếu $ W_q $ và $ W_k $ có hình dạng $ d_k \times d $, trong khi $ W_v $ có hình dạng $ d_v \times d $.

(Lưu ý rằng $ d $ biểu thị kích thước của mỗi vector từ, tức là $ x $.)

Vì chúng ta đang tính tích vô hướng giữa các vector truy vấn và khóa, hai vector này phải chứa cùng số phần tử (tức là $ d_q = d_k $). Tuy nhiên, số phần tử trong vector giá trị $ v^{(i)} $, yếu tố quyết định kích thước của vector ngữ cảnh kết quả, là tùy ý.

Do đó, trong phần hướng dẫn mã nguồn tiếp theo, chúng ta sẽ đặt $ d_q = d_k = 24 $ và sử dụng $ d_v = 28 $, khởi tạo các ma trận chiếu như sau:

In [5]:
torch.manual_seed(123)

d = embedded_sentence.shape[1]
print("d = ",d)
d_q, d_k, d_v = 24, 24, 28

W_query = torch.nn.Parameter(torch.rand(d_q, d))
print("shape W_query = ",W_query.shape)
W_key = torch.nn.Parameter(torch.rand(d_k, d))
print("shape W_key = ",W_key.shape)
W_value = torch.nn.Parameter(torch.rand(d_v, d))
print("Shape W_value = ",W_value.shape)

d =  16
shape W_query =  torch.Size([24, 16])
shape W_key =  torch.Size([24, 16])
Shape W_value =  torch.Size([28, 16])


**Tính toán Trọng số Attention Chưa Chuẩn hóa**

Bây giờ, giả sử chúng ta muốn tính vector attention cho phần tử đầu vào thứ hai – phần tử đầu vào thứ hai này đóng vai trò là truy vấn (query) ở đây:

<div style="text-align: center;">
    <img src="https://sebastianraschka.com/images/blog/2023/self-attention-from-scratch/query.png" alt="Query Illustration" width="300">
</div>

In [6]:
embedded_sentence.shape

torch.Size([6, 16])

In [7]:
x_2 = embedded_sentence[1]
query_2 = W_query.matmul(x_2)
key_2 = W_key.matmul(x_2)
value_2 = W_value.matmul(x_2)

print(query_2.shape)
print(key_2.shape)
print(value_2.shape)

torch.Size([24])
torch.Size([24])
torch.Size([28])


Sau đó, chúng ta có thể tổng quát hóa điều này để tính toán các phần tử khóa (key) và giá trị (value) còn lại cho tất cả các đầu vào, vì chúng ta sẽ cần chúng ở bước tiếp theo khi tính trọng số attention chưa chuẩn hóa $\omega$:

In [8]:
keys = W_key.matmul(embedded_sentence.T).T
values = W_value.matmul(embedded_sentence.T).T

print("keys.shape:", keys.shape)
print("values.shape:", values.shape)

keys.shape: torch.Size([6, 24])
values.shape: torch.Size([6, 28])


Bây giờ, khi đã có tất cả các khóa (key) và giá trị (value) cần thiết, chúng ta có thể tiến hành bước tiếp theo và tính toán trọng số attention chưa chuẩn hóa $ \omega $, được minh họa trong hình dưới đây:
<div style="text-align: center;">
    <img src="https://sebastianraschka.com/images/blog/2023/self-attention-from-scratch/omega.png" alt="Omega Illustration" width="500">
</div>

Như minh họa trong hình trên, chúng ta tính $ \omega_{i,j} $ dưới dạng tích vô hướng giữa các chuỗi truy vấn (query) và khóa (key), $ \omega_{ij} = q(i)^\top k(j) $.

Ví dụ, chúng ta có thể tính trọng số attention chưa chuẩn hóa cho truy vấn và phần tử đầu vào thứ 5 (tương ứng với vị trí chỉ mục 4) như sau:


In [9]:
omega_24 = query_2.dot(keys[4])
print(omega_24)

tensor(35.7423, grad_fn=<DotBackward0>)


Vì chúng ta sẽ cần những giá trị này để tính điểm attention sau đó, hãy tính các giá trị $\omega$ cho tất cả các token đầu vào như đã minh họa trong hình trước:

In [10]:
omega_2 = query_2.matmul(keys.T)
print(omega_2)

tensor([ -5.2174,  30.6948,  74.9203, -73.7313,  35.7423,  -4.6914],
       grad_fn=<SqueezeBackward4>)


**Tính toán Điểm Attention**

Bước tiếp theo trong cơ chế tự chú ý (self-attention) là chuẩn hóa các trọng số attention chưa chuẩn hóa $ \omega $ để thu được các trọng số attention chuẩn hóa $ \alpha $ bằng cách áp dụng hàm softmax. Ngoài ra, $ \frac{1}{\sqrt{d_k}} $ được sử dụng để chia tỷ lệ $ \omega $ trước khi chuẩn hóa nó qua hàm softmax, như minh họa dưới đây:

<div style="text-align: center;">
    <img src="https://sebastianraschka.com/images/blog/2023/self-attention-from-scratch/attention-scores.png" alt="Attention Scores" width="600">
</div>

Việc chia tỷ lệ bởi $ d_k $ đảm bảo rằng độ dài Euclid của các vector trọng số sẽ ở cùng một bậc độ lớn, giúp ngăn các trọng số attention không trở nên quá nhỏ hoặc quá lớn, điều này có thể dẫn đến bất ổn số học hoặc ảnh hưởng đến khả năng hội tụ của mô hình trong quá trình huấn luyện.

Trong mã nguồn, chúng ta có thể triển khai tính toán các trọng số attention như sau:

In [11]:
import torch.nn.functional as F

attention_weights_2 = F.softmax(omega_2 / d_k**0.5, dim=0)
print(attention_weights_2)

tensor([7.8632e-08, 1.2001e-04, 9.9954e-01, 6.6350e-14, 3.3626e-04, 8.7545e-08],
       grad_fn=<SoftmaxBackward0>)


Cuối cùng, bước cuối cùng là tính toán vector ngữ cảnh $ z^{(2)} $, đây là phiên bản có trọng số attention của đầu vào truy vấn ban đầu $ x^{(2)} $, bao gồm tất cả các phần tử đầu vào khác làm ngữ cảnh của nó thông qua các trọng số attention:

<div style="text-align: center;">
    <img src="https://sebastianraschka.com/images/blog/2023/self-attention-from-scratch/context-vector.png" alt="Context Vector" width="500">
</div>

Trong mã nguồn, điều này được thực hiện như sau:

In [12]:
context_vector_2 = attention_weights_2.matmul(values)

print(context_vector_2.shape)
print(context_vector_2.detach())

torch.Size([28])
tensor([-4.1765, -1.6436, -1.9632, -1.6638, -1.0214, -5.0427, -1.4345, -3.0579,
        -1.3729, -1.0158, -0.9395, -2.5402, -2.1344, -1.8692, -1.9991, -3.7605,
        -3.8735, -3.1365, -2.1636, -3.0947, -3.7100, -1.8679, -1.8868, -1.7020,
        -1.4043, -4.1588, -3.5308, -1.8186])


Lưu ý rằng vector đầu ra này có nhiều chiều hơn $( d_v = 28 )$ so với vector đầu vào ban đầu $( d = 16 )$ vì chúng ta đã đặt $ d_v > d $ trước đó; tuy nhiên, việc chọn kích thước embedding là tùy ý.

**Multi-Head Attention**

Trong hình đầu tiên ở đầu bài viết, chúng ta đã thấy rằng transformer sử dụng một module gọi là *multi-head attention*. Điều này liên quan như thế nào đến cơ chế tự chú ý (self-attention) – scaled-dot product attention – mà chúng ta đã tìm hiểu ở trên?

Trong scaled dot-product attention, chuỗi đầu vào được biến đổi thông qua ba ma trận đại diện cho truy vấn (query), khóa (key), và giá trị (value). Ba ma trận này có thể được coi là một "đầu attention" (attention head) trong ngữ cảnh của multi-head attention. Hình dưới đây tóm tắt về một attention head mà chúng ta đã đề cập trước đó:

<div style="text-align: center;">
    <img src="https://sebastianraschka.com/images/blog/2023/self-attention-from-scratch/single-head.png" alt="Single Attention Head" width="400">
</div>

Như tên gọi, multi-head attention bao gồm nhiều head (đầu) attention như vậy, mỗi đầu gồm các ma trận truy vấn (query), khóa (key), và giá trị (value). Khái niệm này tương tự như việc sử dụng nhiều kernel trong mạng neural tích chập (CNN).

<div style="text-align: center;">
    <img src="https://sebastianraschka.com/images/blog/2023/self-attention-from-scratch/multi-head.png" alt="Multi-Head Attention" width="400">
</div>

Để minh họa điều này trong mã nguồn, giả sử chúng ta có 3 đầu attention, do đó, chúng ta mở rộng các ma trận trọng số từ kích thước $ d' \times d $ thành $3 \times d' \times d $:

In [22]:
h = 3
multihead_W_query = torch.nn.Parameter(torch.rand(h, d_q, d))
multihead_W_key = torch.nn.Parameter(torch.rand(h, d_k, d))
multihead_W_value = torch.nn.Parameter(torch.rand(h, d_v, d))


Kết quả là mỗi phần tử query bây giờ có kích thước $ 3 \times d_q $, với $ d_q = 24 $ (ở đây, chúng ta sẽ tập trung vào phần tử thứ 3 tương ứng với vị trí index 2):

In [23]:
multihead_query_2 = multihead_W_query.matmul(x_2)
print(multihead_query_2.shape)

torch.Size([3, 24])



Sau đó, chúng ta có thể lấy các giá trị (value) và khóa (key) theo cách tương tự:


In [24]:
multihead_key_2 = multihead_W_key.matmul(x_2)
multihead_value_2 = multihead_W_value.matmul(x_2)


Các phần tử key và value này hiện là đặc trưng cho phần tử query. Nhưng, giống như trước đó, chúng ta cũng sẽ cần các value và key cho các phần tử khác trong chuỗi để tính toán điểm attention cho query. Chúng ta có thể thực hiện điều này bằng cách mở rộng các embedding của chuỗi đầu vào thành kích thước 3, tức là bằng với số lượng đầu attention:

In [16]:
stacked_inputs = embedded_sentence.T.repeat(3, 1, 1)
print(stacked_inputs.shape)

torch.Size([3, 16, 6])


In [38]:
stacked_inputs[0] == stacked_inputs[1]

tensor([[True, True, True, True, True, True],
        [True, True, True, True, True, True],
        [True, True, True, True, True, True],
        [True, True, True, True, True, True],
        [True, True, True, True, True, True],
        [True, True, True, True, True, True],
        [True, True, True, True, True, True],
        [True, True, True, True, True, True],
        [True, True, True, True, True, True],
        [True, True, True, True, True, True],
        [True, True, True, True, True, True],
        [True, True, True, True, True, True],
        [True, True, True, True, True, True],
        [True, True, True, True, True, True],
        [True, True, True, True, True, True],
        [True, True, True, True, True, True]])


Bây giờ, chúng ta có thể tính toán tất cả các khóa và giá trị bằng cách sử dụng `torch.bmm()` (phép nhân ma trận theo batch):

In [17]:
multihead_keys = torch.bmm(multihead_W_key, stacked_inputs)
multihead_values = torch.bmm(multihead_W_value, stacked_inputs)
print("multihead_keys.shape:", multihead_keys.shape)
print("multihead_values.shape:", multihead_values.shape)

multihead_keys.shape: torch.Size([3, 24, 6])
multihead_values.shape: torch.Size([3, 28, 6])


Chúng ta hiện có các tensor biểu diễn ba đầu attention ở chiều đầu tiên. Chiều thứ ba và thứ hai lần lượt biểu thị số từ và kích thước embedding. Để làm cho các giá trị và khóa dễ hiểu hơn, chúng ta sẽ hoán đổi chiều thứ hai và thứ ba, kết quả là các tensor có cùng cấu trúc chiều như chuỗi đầu vào ban đầu, `embedded_sentence`:

In [18]:

multihead_keys = multihead_keys.permute(0, 2, 1)
multihead_values = multihead_values.permute(0, 2, 1)
print("multihead_keys.shape:", multihead_keys.shape)
print("multihead_values.shape:", multihead_values.shape)

multihead_keys.shape: torch.Size([3, 6, 24])
multihead_values.shape: torch.Size([3, 6, 28])


Sau đó, chúng ta thực hiện các bước tương tự như trước để tính toán trọng số attention chưa chuẩn hóa $ \omega $ và trọng số attention $ \alpha $, sau đó tính toán softmax có chia tỷ lệ để thu được vector ngữ cảnh $ z $ kích thước $ h \times d_v $ (ở đây là $ 3 \times d_v $) cho phần tử đầu vào $ x^{(2)} $.

## Cross-Attention

Trong đoạn code phía trên, chúng ta đã thiết lập $d_q=d_k=24$ và $d_v=28$. Nói cách khác, chúng ta sử dụng cùng kích thước cho các chuỗi truy vấn (query) và khóa (key). Mặc dù ma trận giá trị $\boldsymbol{W_v}
 $ thường được chọn có cùng kích thước với ma trận query và value (như trong lớp MultiHeadAttention của PyTorch), chúng ta có thể chọn kích thước tùy ý cho các giá trị.

Do các kích thước đôi khi khó theo dõi, hãy tóm tắt mọi thứ mà chúng ta đã đề cập ở trên trong hình dưới đây, minh họa các kích thước tensor khác nhau cho một attention head đơn.

<div style="text-align: center;">
    <img src="https://sebastianraschka.com/images/blog/2023/self-attention-from-scratch/summary.png" alt="Summary of Tensor Sizes for Single Attention Head" width="600">
</div>

Hình minh họa trên tương ứng với cơ chế tự chú ý (self-attention) được sử dụng trong transformers. Một biến thể cụ thể của cơ chế attention này mà chúng ta chưa đề cập là *cross-attention*.

<div style="text-align: center;">
    <img src="https://sebastianraschka.com/images/blog/2023/self-attention-from-scratch/cross-attention.png" alt="Cross Attention Mechanism" width="400">
</div>

Vậy cross-attention là gì, và nó khác với self-attention như thế nào?

Trong self-attention, chúng ta chỉ làm việc với cùng một chuỗi đầu vào. Trong cross-attention, chúng ta kết hợp hoặc trộn hai chuỗi đầu vào khác nhau. Trong kiến trúc transformer gốc, đó là chuỗi được trả về từ mô-đun encoder ở bên trái và chuỗi đầu vào được xử lý bởi phần decoder ở bên phải.

Lưu ý rằng trong cross-attention, hai chuỗi đầu vào $x_1$ và $x_2$ có thể có số lượng phần tử khác nhau. Tuy nhiên, các kích thước nhúng của chúng phải khớp.

Hình dưới đây minh họa khái niệm cross-attention. Nếu chúng ta đặt $x_1=x_2$, điều này tương đương với self-attention.

<div style="text-align: center;">
    <img src="https://sebastianraschka.com/images/blog/2023/self-attention-from-scratch/cross-attention-summary.png" alt="Cross Attention Summary" width="400">
</div>

(Lưu ý rằng các truy vấn thường đến từ decoder, còn các khóa và giá trị thường đến từ encoder.)

Điều này hoạt động thế nào trong mã? Trước đây, khi chúng ta triển khai cơ chế self-attention ở phần đầu của bài viết, chúng ta đã sử dụng đoạn mã sau để tính toán truy vấn của phần tử đầu vào thứ hai cùng với tất cả các khóa và giá trị như sau:


In [19]:
torch.manual_seed(123)

d = embedded_sentence.shape[1]
print("embedded_sentence.shape:", embedded_sentence.shape)

d_q, d_k, d_v = 24, 24, 28

W_query = torch.rand(d_q, d)
W_key = torch.rand(d_k, d)
W_value = torch.rand(d_v, d)

x_2 = embedded_sentence[1]
query_2 = W_query.matmul(x_2)
print("query.shape", query_2.shape)

keys = W_key.matmul(embedded_sentence.T).T
values = W_value.matmul(embedded_sentence.T).T

print("keys.shape:", keys.shape)
print("values.shape:", values.shape)

embedded_sentence.shape: torch.Size([6, 16])
query.shape torch.Size([24])
keys.shape: torch.Size([6, 24])
values.shape: torch.Size([6, 28])


Trong cross-attention, điểm khác biệt duy nhất là chúng ta có một chuỗi đầu vào thứ hai, ví dụ, một câu thứ hai với 8 phần tử đầu vào thay vì 6. Giả sử đây là một câu với 8 token.

In [20]:
embedded_sentence_2 = torch.rand(8, 16) # 2nd input sequence

keys = W_key.matmul(embedded_sentence_2.T).T
values = W_value.matmul(embedded_sentence_2.T).T

print("keys.shape:", keys.shape)
print("values.shape:", values.shape)

keys.shape: torch.Size([8, 24])
values.shape: torch.Size([8, 28])


Lưu ý rằng so với self-attention, các khóa (keys) và giá trị (values) giờ có 8 hàng thay vì 6. Mọi thứ khác vẫn giữ nguyên.

Chúng ta đã nói nhiều về transformers ngôn ngữ ở phần trên. Trong kiến trúc transformer gốc, cross-attention trở nên hữu ích khi chuyển từ một câu đầu vào sang một câu đầu ra trong ngữ cảnh dịch ngôn ngữ. Câu đầu vào đại diện cho một chuỗi đầu vào, và bản dịch đại diện cho chuỗi đầu vào thứ hai (hai câu có thể có số lượng từ khác nhau).

Một mô hình phổ biến khác sử dụng cross-attention là Stable Diffusion. Stable Diffusion áp dụng cross-attention giữa hình ảnh được tạo ra trong mô hình U-Net và các lệnh văn bản (text prompts) được dùng làm điều kiện, như mô tả trong bài báo gốc *High-Resolution Image Synthesis with Latent Diffusion Models* – bài báo gốc mô tả mô hình Stable Diffusion, sau đó được Stability AI ứng dụng để phát triển mô hình Stable Diffusion phổ biến.

<div style="text-align: center;">
    <img src="https://sebastianraschka.com/images/blog/2023/self-attention-from-scratch/diffusion.png" alt="Stable Diffusion Cross Attention" width="400">
</div>

**Kết luận**  
Trong bài viết này, chúng ta đã xem cách self-attention hoạt động thông qua cách tiếp cận mã hóa từng bước. Sau đó, chúng ta mở rộng khái niệm này sang multi-head attention, một thành phần được sử dụng rộng rãi trong các transformer ngôn ngữ lớn. Sau khi thảo luận về self-attention và multi-head attention, chúng ta đã giới thiệu thêm một khái niệm nữa: cross-attention, một biến thể của self-attention mà chúng ta có thể áp dụng giữa hai chuỗi khác nhau. Đây là rất nhiều thông tin cần nắm bắt, vì vậy hãy để việc huấn luyện một mạng neural sử dụng khối multi-head attention này cho bài viết trong tương lai.

Nguồn: [Understanding and Coding the Self-Attention Mechanism of Large Language Models From Scratch](https://sebastianraschka.com/blog/2023/self-attention-from-scratch.html)